# Mask R-CNN - Train on Pantograph Dataset


This notebook shows how to train Mask R-CNN on your own dataset. To keep things simple we use a synthetic dataset of shapes (squares, triangles, and circles) which enables fast training. You'd still need a GPU, though, because the network backbone is a Resnet101, which would be too slow to train on a CPU. On a GPU, you can start to get okay-ish results in a few minutes, and good results in less than an hour.

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Add root to path 
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

# Import Mask RCNN
# from mrcnn.config import Config
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

# Import pantogrograph class
from dev import pantograph

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "models")

# Set path to root of images. 
DATA_DIR = os.path.join(ROOT_DIR, "datasets/pantograph")


print("Using Root dir:",ROOT_DIR)
print("Using Model dir:",MODEL_DIR)
print("Using Data dir:",DATA_DIR)

Using TensorFlow backend.


I've been imported
Using Root dir: /home/jupyter/GCP_Test
Using Model dir: /home/jupyter/GCP_Test/models
Using Data dir: /home/jupyter/GCP_Test/datasets/pantograph


## Load Configuration

In [2]:
# Load class config and overwrite values as needed
class InferenceConfig(pantograph.PantographConfig):
    pass
#     MASK_POOL_SIZE = 14
#     KEYPOINT_MASK_POOL_SIZE = 7
#     POOL_SIZE =
#     BACKBONE = "resnet50"
#     STEPS_PER_EPOCH = 150
#     RPN_TRAIN_ANCHORS_PER_IMAGE = 200
#     TRAIN_ROIS_PER_IMAGE = 200
#     STEPS_PER_EPOCH = 2000
#     LEARNING_RATE = .02
#     USE_MINI_MASK = False
#     RPN_ANCHOR_SCALES = (64, 128, 256, 512,1024)

config = InferenceConfig()

config.display()


Configurations Superlee:
BACKBONE                       resnet101
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        50
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0.5
IMAGE_PADDING                  True
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
KEYPOINT_MASK_POOL_SIZE        7
KEYPOINT_MASK_SHAPE            [56, 56]
KEYPOINT_THRESHOLD             0.005
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES        

## Dataset

Create a synthetic dataset

Extend the Dataset class and add a method to load the shapes dataset, `load_shapes()`, and override the following methods:

* load_image()
* load_mask()
* image_reference()

In [3]:
# Load dataset
assert config.NAME == "pantograph"

# Training dataset
train_dataset_keypoints = pantograph.PantographDataset()
train_dataset_keypoints.load_pantograph(DATA_DIR, "train")
train_dataset_keypoints.prepare()

#Validation dataset
val_dataset_keypoints = pantograph.PantographDataset()
val_dataset_keypoints.load_pantograph(DATA_DIR, "val")
val_dataset_keypoints.prepare()

print("Train Keypoints Image Count: {}".format(len(train_dataset_keypoints.image_ids)))
print("Train Keypoints Class Count: {}".format(train_dataset_keypoints.num_classes))
for i, info in enumerate(train_dataset_keypoints.class_info):
    print("{:3}. {:50}".format(i, info['name']))

print("Val Keypoints Image Count: {}".format(len(val_dataset_keypoints.image_ids)))
print("Val Keypoints Class Count: {}".format(val_dataset_keypoints.num_classes))
for i, info in enumerate(val_dataset_keypoints.class_info):
    print("{:3}. {:50}".format(i, info['name']))

loading annotations into memory...
Done (t=0.63s)
creating index...
index created!
Skeleton: (5, 2)
Keypoint names: (6,)
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
Skeleton: (5, 2)
Keypoint names: (6,)
Train Keypoints Image Count: 300
Train Keypoints Class Count: 4
  0. BG                                                
  1. front_bar                                         
  2. middle_bar                                        
  3. rear_bar                                          
Val Keypoints Image Count: 30
Val Keypoints Class Count: 4
  0. BG                                                
  1. front_bar                                         
  2. middle_bar                                        
  3. rear_bar                                          


## Create Model

In [4]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", 
                          config=config,
                          model_dir=MODEL_DIR)



# Which weights to start with?
init_with = "last"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that are different due to the different number of classes    
    MODEL_PATH = os.path.join(MODEL_DIR, "mask_rcnn_coco.h5")
    model.load_weights(MODEL_PATH, by_name=True,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])
elif init_with == 'last':
    LOG_DIR = os.path.join(MODEL_DIR, "pantograph20200429T1039")
    MODEL_PATH = os.path.join(LOG_DIR, "mask_rcnn_pantograph_0410.h5")
    model.load_weights(MODEL_PATH, by_name=True,exclude=None)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


In [ ]:
# model.keras_model.summary()
# model.get_trainable_layers()

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [5]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(train_dataset_keypoints, val_dataset_keypoints, 
            learning_rate=config.LEARNING_RATE, #
            epochs=1000, 
            layers="heads")


Starting at epoch 411. LR=0.001

Checkpoint Path: /home/jupyter/GCP_Test/models/pantograph20200429T1039/mask_rcnn_pantograph_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_keypoint_mask_conv1   (TimeDistributed)
mrcnn_keypoint_mask_bn1   (TimeDistributed)
mrcnn_keypoint_mask_conv2   (TimeDistributed)
mrcnn_keypoint_mask_bn2   (TimeDistributed)
mrcnn_keypoint_mask_conv3   (TimeDistributed)
mrcnn_keypoint_mask_bn3   (TimeDistributed)
mrcnn_keypoint_mask_conv4   (TimeDistributed)
mrcnn_keypoint_mask_bn4   (TimeDistributed)
mrcnn_keypoint_mask_conv5   (TimeDistributed)
mrcnn_keypoint_mask_bn5   (TimeDistributed

/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting spars

/usr/local/lib/python3.5/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 412/1000
150/150 [==============================] - 88s 586ms/step - loss: 2.5265 - val_loss: 2.9954


Epoch 00412: saving model to /home/jupyter/GCP_Test/models/pantograph20200429T1039/mask_rcnn_pantograph_0412.h5
Epoch 413/1000
150/150 [==============================] - 68s 453ms/step - loss: 2.5208 - val_loss: 2.9804

Epoch 00413: saving model to /home/jupyter/GCP_Test/models/pantograph20200429T1039/mask_rcnn_pantograph_0413.h5
Epoch 414/1000
150/150 [==============================] - 69s 457ms/step - loss: 2.5229 - val_loss: 2.9618

Epoch 00414: saving model to /home/jupyter/GCP_Test/models/pantograph20200429T1039/mask_rcnn_pantograph_0414.h5
Epoch 415/1000
150/150 [==============================] - 75s 500ms/step - loss: 2.5264 - val_loss: 2.9961

Epoch 00415: saving model to /home/jupyter/GCP_Test/models/pantograph20200429T1039/mask_rcnn_pantograph_0415.h5
Epoch 416/1000
150/150 [==============================] - 70s 468ms/step - loss: 2.5278 - val_loss: 3.1979

Epoch 00416

Process ForkPoolWorker-1:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
Process ForkPoolWorker-2:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.

KeyboardInterrupt: 

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes_test_1.h5")
model.keras_model.save_weights(model_path)